# KG CREATION PIPELINE

This is a tool that allows the creation of a robust ontology-based knowledge graph from unstrctured input. The knowledge graph will be stored in Neo4j.

## FOR REFERENCE:

TKGCon: 
https://medium.com/@researchgraph/automated-knowledge-graph-construction-with-large-language-models-part-2-b107ca8ec5ea

Entity Resolution: 
https://towardsdatascience.com/entity-resolution-identifying-real-world-entities-in-noisy-data-3e8c59f4f41c

### GOALS
1. Multimodal input
2. Self-augmentation from user queries and automated web search
3. Agents and tools

### PROCESS

![TKGCon](Markdown_Images\TKGCon.png)
1. PDF to image extraction and generation of description in unstructured text
2. Text splitting and storage in local vector database
3. Creation of unrefined triples
3. Determination of themes
4. Ontology creation based on themes (Human in-the-loop)
5. Knowledge graph construction 

## INSTALLING DEPENDENCIES

In [3]:
%pip install pdf2image==1.17.0 openai==1.30.1 tiktoken==0.7.0 python-dotenv==1.0.1 pandas==2.2.2 langchain==0.2.11 langchain-community==0.2.10 langchain-openai==0.1.17 langchain-experimental==0.0.63 neo4j==5.22.0 langchain-chroma==

^C
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install langchain-chroma 

  Using cached langchain_chroma-0.1.2-py3-none-any.whl.metadata (1.3 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ---------------------------------------- 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached python_multipart-0.0.9-py3-none-any.whl.metadata (2.5 kB)
  Using cached MarkupSafe-2.1.5-cp312-cp31

  error: subprocess-exited-with-error
  
  × Building wheel for chroma-hnswlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'hnswlib' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib
ERROR: Could not build wheels for chroma-hnswlib, which is required to install pyproject.toml-based projects


In [7]:
%pip show langchain-chroma

Note: you may need to restart the kernel to use updated packages.


## SETUP